In [25]:
import pandas as pd


In [26]:
df_compile_stats = pd.read_csv('data/targeted/compile_stats.csv')
df_goalk_stats = pd.read_csv('data/targeted/goalkeepers_data.csv')


In [ ]:
def calculate_player_stats(df, output_file=r"data\processed\red_method\player_stats.csv"):
    # Фильтруем игроков с амплуа 9 и 10
    df_filtered = df[df['amplua'].isin([9, 10])]
    
    # Группируем по игроку и амплуа, считаем количество игр и суммируем показатели
    player_stats = df_filtered.groupby(['ID player', 'amplua']).agg(
        games=('ID game', 'nunique'),  # Количество уникальных игр
        goals=('goals', 'sum'),
        assists=('assists', 'sum'),
        throws_by=('throws by', 'sum'),
        shot_on_target=('a shot on target', 'sum'),
        blocked_throws=('blocked throws', 'sum'),
        p_m=('p/m', 'sum'),
        total_time=('total time on ice', 'sum')
    ).reset_index()
    
    # Сохраняем в CSV
    player_stats.to_csv(output_file, index=False)
    print(f"Файл сохранен: {output_file}")
    
    return player_stats

def calculate_points(df, coefficient, amplua):
    df_filtered = df[df['amplua'] == amplua].copy()
    
    # Рассчитываем очки по каждому показателю
    for col in ['goals', 'assists', 'throws_by', 'shot_on_target', 'blocked_throws', 'p_m']:
        df_filtered[f'p_{col}'] = ((df_filtered[col] + coefficient * df_filtered['games']) ** 2) / df_filtered['games']
    
    return round(df_filtered, 2)

def process_and_save(df, output_file=r"data\processed\red_method\player_stats_with_points.csv"):
    df_stats = calculate_player_stats(df)
    
    # Вычисляем очки для каждого амплуа с соответствующими коэффициентами
    df_defenders = calculate_points(df_stats, 2/3, 9)
    df_forwards = calculate_points(df_stats, 1/6, 10)
    
    # Объединяем результаты
    df_final = pd.concat([df_defenders, df_forwards])
    
    df_final = df_final.reindex(columns=['ID player', 'amplua', 'goals', 'p_goals', 'assists', 'p_assists',  'throws_by', 'p_throws_by', 'shot_on_target', 'p_shot_on_target', 
                                         'blocked_throws', 'p_blocked_throws', 'p_m', 'p_p_m'])

    # Сохраняем в CSV
    df_final.to_csv(output_file, index=False)
    print(f"Файл сохранен: {output_file}")
    
    return df_final


In [28]:
rezult_rating_red = process_and_save(df_compile_stats)

Файл сохранен: data\processed\red_method\player_stats.csv
Файл сохранен: data\processed\red_method\player_stats_with_points.csv
